In [1]:
param_market = 'US'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_item = {}
list_job = []
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Page'] != '':
        vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
        dict_item[input_.loc[i, 'Item_Number']] = {'vehicle_page': vehicle_page,
                                                   'crawler': 0,
                                                   'dict_vehicle': {},
                                                   'vehicle_1': '',
                                                   'vehicle_2': ''}
        for j in range(vehicle_page):
            list_job.append((i, j))

print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(list_job)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                i, j = list_job[a]

                data = '{"scopedContext":{"catalogDetails":{"itemId":"' + input_.loc[i, 'Item_Number'] + '","categoryId":"' + input_.loc[i, 'Category_Id'] + '","marketplaceId":"' + input_.loc[i, 'Market_Id'] + '"}}}'

                # ua = UA.get_User_Agent_Requests()
                # ua['Content-Type'] = 'application/json'
                ua = {'Content-Type': 'application/json'}

                if param_market == 'US':
                    url = 'https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE':
                    url = 'https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=DE&_stpos=10115&orig_cvip=true'
                elif param_market == 'AU':
                    url = 'https://www.ebay.com.au/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&ul=AU&_stpos=2019&orig_cvip=true'
                elif param_market == 'UK':
                    url = 'https://www.ebay.co.uk/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=GB&_stpos=BS81QU&orig_cvip=true'

                resp = requests.post(url,
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_header = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']
                
                # = = = = = = = = = = = = = = =
                
                col_engine = -1
                if param_market in ['US', 'AU', 'UK']:
                    name_make = 'Make'
                    name_model = 'Model'
                    name_engine = 'Engine'
                    name_year = 'Year'
                elif param_market == 'DE':
                    name_make = 'Marke'
                    name_model = 'Modell'
                    name_engine = 'Motor'
                    name_year = 'Baujahr'

                for k in range(len(list_header)):
                    if name_make in list_header[k]['textSpans'][0]['text']:
                        col_make = k
                    elif name_model in list_header[k]['textSpans'][0]['text']:
                        col_model = k
                    elif name_engine in list_header[k]['textSpans'][0]['text']:
                        col_engine = k
                    elif name_year in list_header[k]['textSpans'][0]['text']:
                        col_year = k
                        
                # = = = = = = = = = = = = = = =
                
                list_row = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

                # = = = = = = = = = = = = = = =
                
                for row in list_row:
                    make = row['cells'][col_make]['textSpans'][0]['text'].strip()
                    model = row['cells'][col_model]['textSpans'][0]['text'].strip()
                    if col_engine == -1:
                        engine = ''
                    else:
                        engine = row['cells'][col_engine]['textSpans'][0]['text'].strip()
                    if param_market in ['US', 'AU', 'UK']:
                        list_year = [int(row['cells'][col_year]['textSpans'][0]['text'].strip())]
                    elif param_market == 'DE':
                        list_year = [int(year.split('/').strip()) for year in row['cells'][col_year]['textSpans'][0]['text'].split('-')]

                    if make not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make] = {model: {engine: list_year}}
                    else:
                        if model not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model] = {engine: list_year}
                        else:
                            if engine not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list_year
                            else:
                                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] += list_year

                    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = sorted(list(set(dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine])))

                dict_item[input_.loc[i, 'Item_Number']]['crawler'] += 1

                # = = = = = = = = = = = = = = =
                
                if dict_item[input_.loc[i, 'Item_Number']]['vehicle_page'] == dict_item[input_.loc[i, 'Item_Number']]['crawler']:
                    list_vehicle = []
                    for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            list_year = []
                            for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                list_year += dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]
                            list_year = list(set(list_year))
                            list_year.sort()

                            year_min = list_year[0]
                            year_max = list_year[-1]
                            if year_min == year_max:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    list_vehicle.sort()

                    dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_vehicle = []
                    for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                year_min = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][0]
                                year_max = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][-1]
                                if year_min == year_max:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                                else:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                    list_vehicle.sort()

                    dict_item[input_.loc[i, 'Item_Number']]['vehicle_2'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                             'Vehicle_1': dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'],
                                             'Vehicle_2': dict_item[input_.loc[i, 'Item_Number']]['vehicle_2']}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                     'Vehicle_Page': input_.loc[i, 'Vehicle_Page'],
                                     'Category_Id': input_.loc[i, 'Category_Id'],
                                     'Market_Id': input_.loc[i, 'Market_Id']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[i, 'Item_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：10780

[ok] - 235266939274
[尝试次数：1] - [剩余数量：10750] - [当前时间：15:28:27]

[ok] - 235266939274
[尝试次数：1] - [剩余数量：10749] - [当前时间：15:28:27]

[ok] - 235266939274
[尝试次数：1] - [剩余数量：10748] - [当前时间：15:28:27]

[ok] - 235266939274
[尝试次数：1] - [剩余数量：10747] - [当前时间：15:28:27]

[ok] - 235266939274
[尝试次数：2] - [剩余数量：10746] - [当前时间：15:28:27]

[ok] - 235266939274
[尝试次数：1] - [剩余数量：10745] - [当前时间：15:28:27]

[ok] - 235266939274
[尝试次数：1] - [剩余数量：10744] - [当前时间：15:28:27]

[ok] - 235266939274
[尝试次数：2] - [剩余数量：10743] - [当前时间：15:28:27]

[ok] - 235266939274
[尝试次数：1] - [剩余数量：10742] - [当前时间：15:28:27]

[ok] - 235266939274
[尝试次数：1] - [剩余数量：10741] - [当前时间：15:28:27]

[ok] - 235266939274
[尝试次数：3] - [剩余数量：10740] - [当前时间：15:28:28]

[ok] - 235266939274
[尝试次数：1] - [剩余数量：10739] - [当前时间：15:28:28]

[ok] - 235266939274
[尝试次数：5] - [剩余数量：10738] - [当前时间：15:28:28]

[ok] - 235266939274
[尝试次数：2] - [剩余数量：10737] - [当前时间：15:28:29]

[ok] - 235266939274
[尝试次数：6] - [剩余数量：10736] - [当前时间：15:28:29]

[ok] - 235266939274
[尝试次数：3] - [剩余数量：10735] 

[ok] - 234091521908
[尝试次数：18] - [剩余数量：10620] - [当前时间：15:28:56]

[ok] - 375099192744
[尝试次数：2] - [剩余数量：10619] - [当前时间：15:28:56]

[ok] - 375099192744
[尝试次数：1] - [剩余数量：10618] - [当前时间：15:28:56]

[ok] - 375099192744
[尝试次数：6] - [剩余数量：10617] - [当前时间：15:28:56]

[ok] - 372315045310
[尝试次数：36] - [剩余数量：10616] - [当前时间：15:28:56]

[ok] - 372315045310
[尝试次数：27] - [剩余数量：10615] - [当前时间：15:28:57]

[ok] - 375099192744
[尝试次数：5] - [剩余数量：10614] - [当前时间：15:28:57]

[ok] - 353577009447
[尝试次数：12] - [剩余数量：10613] - [当前时间：15:28:57]

[ok] - 235266939274
[尝试次数：1] - [剩余数量：10612] - [当前时间：15:28:59]

[ok] - 375099192744
[尝试次数：5] - [剩余数量：10611] - [当前时间：15:28:59]

[ok] - 233147962097
[尝试次数：20] - [剩余数量：10610] - [当前时间：15:28:59]

[ok] - 232572207557
[尝试次数：49] - [剩余数量：10609] - [当前时间：15:28:59]

[ok] - 375099192744
[尝试次数：6] - [剩余数量：10608] - [当前时间：15:28:59]

[ok] - 375099192744
[尝试次数：8] - [剩余数量：10607] - [当前时间：15:29:00]

[ok] - 375099192744
[尝试次数：6] - [剩余数量：10606] - [当前时间：15:29:00]

[ok] - 375099192744
[尝试次数：7] - [剩余数量：10605] - [当前

[ok] - 353578660265
[尝试次数：1] - [剩余数量：10488] - [当前时间：15:29:26]

[ok] - 234091479668
[尝试次数：8] - [剩余数量：10487] - [当前时间：15:29:27]

[ok] - 353578660265
[尝试次数：1] - [剩余数量：10486] - [当前时间：15:29:27]

[ok] - 353578660265
[尝试次数：2] - [剩余数量：10485] - [当前时间：15:29:27]

[ok] - 353522027654
[尝试次数：29] - [剩余数量：10484] - [当前时间：15:29:27]

[ok] - 232780217507
[尝试次数：15] - [剩余数量：10483] - [当前时间：15:29:27]

[ok] - 232780217507
[尝试次数：21] - [剩余数量：10482] - [当前时间：15:29:27]

[ok] - 374864991975
[尝试次数：5] - [剩余数量：10481] - [当前时间：15:29:28]

[ok] - 353578646770
[尝试次数：13] - [剩余数量：10480] - [当前时间：15:29:28]

[ok] - 232780217507
[尝试次数：14] - [剩余数量：10479] - [当前时间：15:29:28]

[ok] - 353578660265
[尝试次数：5] - [剩余数量：10478] - [当前时间：15:29:28]

[ok] - 232780217507
[尝试次数：30] - [剩余数量：10477] - [当前时间：15:29:28]

[ok] - 353578660265
[尝试次数：3] - [剩余数量：10476] - [当前时间：15:29:29]

[ok] - 352565925068
[尝试次数：2] - [剩余数量：10475] - [当前时间：15:29:29]

[ok] - 232780217507
[尝试次数：22] - [剩余数量：10474] - [当前时间：15:29:29]

[ok] - 353578646654
[尝试次数：24] - [剩余数量：10473] - [

[ok] - 353577017852
[尝试次数：1] - [剩余数量：10358] - [当前时间：15:29:58]

[ok] - 353577017852
[尝试次数：3] - [剩余数量：10357] - [当前时间：15:29:58]

[ok] - 353577017852
[尝试次数：9] - [剩余数量：10356] - [当前时间：15:29:58]

[ok] - 232780217507
[尝试次数：31] - [剩余数量：10355] - [当前时间：15:29:58]

[ok] - 353577017852
[尝试次数：1] - [剩余数量：10354] - [当前时间：15:29:58]

[ok] - 353577017852
[尝试次数：1] - [剩余数量：10353] - [当前时间：15:29:59]

[ok] - 353577017852
[尝试次数：14] - [剩余数量：10352] - [当前时间：15:29:59]

[ok] - 352363975568
[尝试次数：48] - [剩余数量：10351] - [当前时间：15:29:59]

[ok] - 352363975568
[尝试次数：46] - [剩余数量：10350] - [当前时间：15:29:59]

[ok] - 353577017852
[尝试次数：2] - [剩余数量：10349] - [当前时间：15:29:59]

[ok] - 353577017852
[尝试次数：10] - [剩余数量：10348] - [当前时间：15:30:00]

[ok] - 353577017852
[尝试次数：7] - [剩余数量：10347] - [当前时间：15:30:00]

[ok] - 353577017852
[尝试次数：2] - [剩余数量：10346] - [当前时间：15:30:00]

[ok] - 353577017852
[尝试次数：4] - [剩余数量：10345] - [当前时间：15:30:00]

[ok] - 353577017852
[尝试次数：28] - [剩余数量：10344] - [当前时间：15:30:00]

[ok] - 353577017852
[尝试次数：15] - [剩余数量：10343] - [当

[ok] - 371671949428
[尝试次数：8] - [剩余数量：10228] - [当前时间：15:30:26]

[ok] - 352562422774
[尝试次数：3] - [剩余数量：10227] - [当前时间：15:30:26]

[ok] - 234091479733
[尝试次数：8] - [剩余数量：10226] - [当前时间：15:30:26]

[ok] - 234041047113
[尝试次数：19] - [剩余数量：10225] - [当前时间：15:30:26]

[ok] - 372318935925
[尝试次数：31] - [剩余数量：10224] - [当前时间：15:30:26]

[ok] - 352562422774
[尝试次数：4] - [剩余数量：10223] - [当前时间：15:30:26]

[ok] - 372550270041
[尝试次数：2] - [剩余数量：10222] - [当前时间：15:30:26]

[ok] - 232788231556
[尝试次数：7] - [剩余数量：10221] - [当前时间：15:30:27]

[ok] - 353465375470
[尝试次数：5] - [剩余数量：10220] - [当前时间：15:30:27]

[ok] - 233087090062
[尝试次数：15] - [剩余数量：10219] - [当前时间：15:30:27]

[ok] - 352366960698
[尝试次数：13] - [剩余数量：10218] - [当前时间：15:30:27]

[ok] - 373072622881
[尝试次数：20] - [剩余数量：10217] - [当前时间：15:30:28]

[ok] - 373651447092
[尝试次数：3] - [剩余数量：10216] - [当前时间：15:30:28]

[ok] - 372320524527
[尝试次数：4] - [剩余数量：10215] - [当前时间：15:30:28]

[ok] - 372145122059
[尝试次数：6] - [剩余数量：10214] - [当前时间：15:30:29]

[ok] - 373603461354
[尝试次数：11] - [剩余数量：10213] - [当前

...

[ok] - 234039583402
[尝试次数：12] - [剩余数量：535] - [当前时间：16:03:40]

[ok] - 234039583402
[尝试次数：2] - [剩余数量：534] - [当前时间：16:03:40]

[ok] - 234039583402
[尝试次数：1] - [剩余数量：533] - [当前时间：16:03:40]

[ok] - 235199330692
[尝试次数：23] - [剩余数量：532] - [当前时间：16:03:41]

[ok] - 234039583402
[尝试次数：7] - [剩余数量：531] - [当前时间：16:03:41]

[ok] - 234039583402
[尝试次数：2] - [剩余数量：530] - [当前时间：16:03:41]

[ok] - 354805616340
[尝试次数：59] - [剩余数量：529] - [当前时间：16:03:41]

[ok] - 234039583402
[尝试次数：1] - [剩余数量：528] - [当前时间：16:03:41]

[ok] - 234039583402
[尝试次数：2] - [剩余数量：527] - [当前时间：16:03:41]

[ok] - 234039583402
[尝试次数：7] - [剩余数量：526] - [当前时间：16:03:41]

[ok] - 234039583402
[尝试次数：2] - [剩余数量：525] - [当前时间：16:03:42]

[ok] - 234039583402
[尝试次数：25] - [剩余数量：524] - [当前时间：16:03:42]

[ok] - 234039583402
[尝试次数：1] - [剩余数量：523] - [当前时间：16:03:42]

[ok] - 234039583402
[尝试次数：1] - [剩余数量：522] - [当前时间：16:03:42]

[ok] - 235199330692
[尝试次数：6] - [剩余数量：521] - [当前时间：16:03:42]

[ok] - 234039583402
[尝试次数：4] - [剩余数量：520] - [当前时间：16:03:42]

[ok] - 234039583402


[ok] - 374555442507
[尝试次数：8] - [剩余数量：401] - [当前时间：16:04:06]

[ok] - 234039583402
[尝试次数：20] - [剩余数量：400] - [当前时间：16:04:06]

[ok] - 235383977305
[尝试次数：1] - [剩余数量：399] - [当前时间：16:04:07]

[ok] - 233994913881
[尝试次数：12] - [剩余数量：398] - [当前时间：16:04:07]

[ok] - 235383977305
[尝试次数：2] - [剩余数量：397] - [当前时间：16:04:07]

[ok] - 374140809421
[尝试次数：2] - [剩余数量：396] - [当前时间：16:04:07]

[ok] - 234595755649
[尝试次数：4] - [剩余数量：395] - [当前时间：16:04:07]

[ok] - 235383977305
[尝试次数：1] - [剩余数量：394] - [当前时间：16:04:08]

[ok] - 375334609087
[尝试次数：10] - [剩余数量：393] - [当前时间：16:04:08]

[ok] - 354236081927
[尝试次数：3] - [剩余数量：392] - [当前时间：16:04:08]

[ok] - 234039583402
[尝试次数：47] - [剩余数量：391] - [当前时间：16:04:09]

[ok] - 374492526977
[尝试次数：17] - [剩余数量：390] - [当前时间：16:04:09]

[ok] - 234039583402
[尝试次数：23] - [剩余数量：389] - [当前时间：16:04:09]

[ok] - 354575773053
[尝试次数：3] - [剩余数量：388] - [当前时间：16:04:09]

[ok] - 354575773053
[尝试次数：4] - [剩余数量：387] - [当前时间：16:04:09]

[ok] - 234257944369
[尝试次数：11] - [剩余数量：386] - [当前时间：16:04:10]

[ok] - 3741408094

[ok] - 233903151135
[尝试次数：15] - [剩余数量：267] - [当前时间：16:04:33]

[ok] - 354638862136
[尝试次数：25] - [剩余数量：266] - [当前时间：16:04:33]

[ok] - 351773500545
[尝试次数：45] - [剩余数量：265] - [当前时间：16:04:33]

[ok] - 234803747298
[尝试次数：5] - [剩余数量：264] - [当前时间：16:04:33]

[ok] - 374237455508
[尝试次数：13] - [剩余数量：263] - [当前时间：16:04:33]

[ok] - 234616717149
[尝试次数：1] - [剩余数量：262] - [当前时间：16:04:34]

[ok] - 375398502408
[尝试次数：9] - [剩余数量：261] - [当前时间：16:04:34]

[ok] - 354638862136
[尝试次数：15] - [剩余数量：260] - [当前时间：16:04:34]

[ok] - 234595755618
[尝试次数：13] - [剩余数量：259] - [当前时间：16:04:34]

[ok] - 354130611815
[尝试次数：3] - [剩余数量：258] - [当前时间：16:04:34]

[ok] - 234616717149
[尝试次数：1] - [剩余数量：257] - [当前时间：16:04:35]

[ok] - 374466869822
[尝试次数：23] - [剩余数量：256] - [当前时间：16:04:35]

[ok] - 234883508748
[尝试次数：29] - [剩余数量：255] - [当前时间：16:04:35]

[ok] - 234595755618
[尝试次数：10] - [剩余数量：254] - [当前时间：16:04:35]

[ok] - 353523633762
[尝试次数：3] - [剩余数量：253] - [当前时间：16:04:36]

[ok] - 372756656735
[尝试次数：1] - [剩余数量：252] - [当前时间：16:04:36]

[ok] - 35352363

[ok] - 353743952901
[尝试次数：4] - [剩余数量：132] - [当前时间：16:05:00]

[ok] - 354256700938
[尝试次数：1] - [剩余数量：131] - [当前时间：16:05:00]

[ok] - 374466869726
[尝试次数：2] - [剩余数量：130] - [当前时间：16:05:00]

[ok] - 354558054766
[尝试次数：16] - [剩余数量：129] - [当前时间：16:05:00]

[ok] - 355371307117
[尝试次数：2] - [剩余数量：128] - [当前时间：16:05:01]

[ok] - 354558054766
[尝试次数：18] - [剩余数量：127] - [当前时间：16:05:01]

[ok] - 374466869726
[尝试次数：2] - [剩余数量：126] - [当前时间：16:05:01]

[ok] - 354256700938
[尝试次数：3] - [剩余数量：125] - [当前时间：16:05:02]

[ok] - 234624888250
[尝试次数：1] - [剩余数量：124] - [当前时间：16:05:02]

[ok] - 354558054766
[尝试次数：25] - [剩余数量：123] - [当前时间：16:05:02]

[ok] - 233071983715
[尝试次数：2] - [剩余数量：122] - [当前时间：16:05:02]

[ok] - 374466869726
[尝试次数：5] - [剩余数量：121] - [当前时间：16:05:02]

[ok] - 234624888250
[尝试次数：2] - [剩余数量：120] - [当前时间：16:05:03]

[ok] - 355581493916
[尝试次数：12] - [剩余数量：119] - [当前时间：16:05:03]

[ok] - 233872739238
[尝试次数：2] - [剩余数量：118] - [当前时间：16:05:03]

[ok] - 355371297356
[尝试次数：17] - [剩余数量：117] - [当前时间：16:05:03]

[ok] - 375314299502

[ok] - 352594707707
[尝试次数：9] - [剩余数量：0] - [当前时间：16:05:32]

[ok] - 352363277988
[尝试次数：9] - [剩余数量：0] - [当前时间：16:05:32]

[ok] - 234091494660
[尝试次数：5] - [剩余数量：0] - [当前时间：16:05:32]

[ok] - 235493622180
[尝试次数：53] - [剩余数量：0] - [当前时间：16:05:33]

[ok] - 234927458690
[尝试次数：14] - [剩余数量：0] - [当前时间：16:05:33]

[ok] - 355549108262
[尝试次数：77] - [剩余数量：0] - [当前时间：16:05:33]

[ok] - 374177228711
[尝试次数：17] - [剩余数量：0] - [当前时间：16:05:33]

[ok] - 372435192537
[尝试次数：14] - [剩余数量：0] - [当前时间：16:05:33]

[ok] - 234091494660
[尝试次数：4] - [剩余数量：0] - [当前时间：16:05:34]

[ok] - 372315064101
[尝试次数：18] - [剩余数量：0] - [当前时间：16:05:34]

[ok] - 233072098050
[尝试次数：5] - [剩余数量：0] - [当前时间：16:05:35]

[ok] - 232785880060
[尝试次数：20] - [剩余数量：0] - [当前时间：16:05:35]

[ok] - 234091494660
[尝试次数：10] - [剩余数量：0] - [当前时间：16:05:35]

[ok] - 372533893044
[尝试次数：17] - [剩余数量：0] - [当前时间：16:05:35]

[ok] - 374177228711
[尝试次数：22] - [剩余数量：0] - [当前时间：16:05:35]

[ok] - 352594707707
[尝试次数：20] - [剩余数量：0] - [当前时间：16:05:36]

[ok] - 232797372035
[尝试次数：23] - [剩余数量：0] - [当